# Face Timelapse Portrait Generator

This notebook demonstrates the refactored face alignment pipeline using the new modular structure.

## Setup and Configuration

In [ ]:
import yaml
import matplotlib.pyplot as plt
import cv2
from PIL import Image

from src.metadata_manager import MetadataManager
from src.image_processor import ImageProcessor
from src.video_generator import VideoGenerator
from src.average_image import AverageImageGenerator
from src.google_photos import GooglePhotosDownloader

# Load configuration
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)

print("Configuration loaded successfully")

## Initialize Components

In [ ]:
# Initialize metadata manager
metadata = MetadataManager(config['paths']['metadata'])

# Initialize image processor
processor = ImageProcessor(
    landmarks_model_path=config['paths']['landmarks_model'],
    face_width=config['alignment']['face_width'],
    face_height=config['alignment']['face_height'],
    left_eye_position=tuple(config['alignment']['left_eye_position']),
    resize_width=config['alignment']['resize_width'],
    upsample_times=config['detection']['upsample_times'],
    auto_select_largest=config['detection']['auto_select_largest'],
    verbose=True
)

print("Components initialized")

## Process a Single Image (for testing)

In [ ]:
# Pick a test image
test_image_path = 'original_faces/test_image.jpg'  # Change to your actual image path
output_path = 'test_aligned.jpg'

# Process single image
success = processor.process_single_image(
    test_image_path,
    output_path,
    metadata=metadata,
    skip_existing=False
)

if success:
    # Display results
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
    
    # Original
    original = cv2.imread(test_image_path)
    original_rgb = cv2.cvtColor(original, cv2.COLOR_BGR2RGB)
    ax1.imshow(original_rgb)
    ax1.set_title('Original')
    ax1.axis('off')
    
    # Aligned
    aligned = cv2.imread(output_path)
    aligned_rgb = cv2.cvtColor(aligned, cv2.COLOR_BGR2RGB)
    ax2.imshow(aligned_rgb)
    ax2.set_title('Aligned')
    ax2.axis('off')
    
    plt.tight_layout()
    plt.show()
else:
    print("Failed to process image")

## Process All Images in Directory

In [ ]:
# Process entire directory
successful, failed = processor.process_directory(
    config['paths']['original_faces'],
    config['paths']['aligned_faces'],
    metadata=metadata,
    skip_existing=True
)

print(f"\nResults:")
print(f"  Successful: {successful}")
print(f"  Failed: {failed}")

## Generate Videos

In [ ]:
# Get images with dates
date_images = ImageProcessor.get_images_with_dates(
    config['paths']['aligned_faces'],
    metadata
)

print(f"Found {len(date_images)} images with dates")

# Initialize video generator
video_gen = VideoGenerator(
    fps=config['video']['fps'],
    codec=config['video']['codec'],
    show_date_overlay=config['video']['show_date_overlay'],
    verbose=True
)

# Create overall video
video_gen.create_overall_video(
    f"{config['paths']['videos']}/timelapse.mp4",
    date_images
)

# Create yearly videos
if config['video']['generate_yearly_videos']:
    videos_created = video_gen.create_videos_by_year(
        config['paths']['videos'],
        date_images
    )
    print(f"Created {videos_created} yearly videos")

## Generate Average Images

In [ ]:
# Initialize average image generator
avg_gen = AverageImageGenerator(verbose=True)

# Generate all averages
counts = avg_gen.generate_all_averages(
    date_images,
    config['paths']['average_images'],
    generate_overall=config['average_images']['generate_overall'],
    generate_by_year=config['average_images']['generate_by_year'],
    generate_by_quarter=config['average_images']['generate_by_quarter'],
    generate_by_month=config['average_images']['generate_by_month'],
    min_images=config['average_images']['min_images']
)

print("\nAverage images generated:")
print(f"  Overall: {counts['overall']}")
print(f"  Yearly: {counts['year']}")
print(f"  Quarterly: {counts['quarter']}")
print(f"  Monthly: {counts['month']}")

## Display Average Image

In [ ]:
# Display overall average
overall_avg_path = f"{config['paths']['average_images']}/overall_average_image.jpg"
avg_image = cv2.imread(overall_avg_path)
avg_rgb = cv2.cvtColor(avg_image, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(8, 8))
plt.imshow(avg_rgb)
plt.title('Overall Average Face')
plt.axis('off')
plt.show()

## Google Photos Integration (Optional)

In [ ]:
# Only run if Google Photos is configured
if config['google_photos']['enabled']:
    downloader = GooglePhotosDownloader(verbose=True)
    
    # Sync album
    downloaded = downloader.sync_album(
        config['google_photos']['album_name'],
        config['paths']['original_faces'],
        metadata
    )
    
    print(f"Downloaded {downloaded} new photos")
else:
    print("Google Photos sync is disabled in config.yaml")

## Metadata Analysis

In [ ]:
# View failed images
failed_images = metadata.get_failed_images()

print(f"Total failed images: {len(failed_images)}\n")

# Group by category
by_category = {}
for img, info in failed_images.items():
    category = info.get('category', 'unknown')
    if category not in by_category:
        by_category[category] = []
    by_category[category].append(img)

for category, images in by_category.items():
    print(f"{category}: {len(images)} images")
    for img in images[:5]:  # Show first 5
        print(f"  - {img}")
    if len(images) > 5:
        print(f"  ... and {len(images) - 5} more")
    print()